In [76]:
import pandas as pd
import pymysql
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce

In [77]:
# conectarse a bbdd mysql
with open ('password.txt') as f:
   pwd=str(f.readline()).strip()

conexion = pymysql.connect(host='localhost',
                           database = 'henry_pi',
                           user= 'root',
                           password=pwd)

cursor = conexion.cursor()

In [78]:
# se le pasa el nombre de una tabla contenida en una base de datos a la cual se esta conectado
# devuelve un dataframe
def extraer_datos(tabla:str):
    cursor.execute("SELECT * FROM {}".format(tabla))
    rows = cursor.fetchall()
    headers = [column[0] for column in cursor.description]
    df= pd.DataFrame(rows,columns=headers)
    return df


In [79]:
# extraer datos de bbdd
historico_velocidad_nacional=extraer_datos('historico_velocidad_nacional')
ingresos_trimestre=extraer_datos('ingresos_trimestre')
internet_accesos_velocidad_provincias=extraer_datos('internet_accesos_velocidad_provincias')
internet_penetracion_hogares=extraer_datos('internet_penetracion_hogares')
internet_tipo_nacional=extraer_datos('internet_tipo_nacional')
internet_tipo_provincias=extraer_datos('internet_tipo_provincias')
localidades_servicios=extraer_datos('localidades_servicios')

In [80]:
# unir los dataframes segun la estructura de los datos
nacionales = [historico_velocidad_nacional,ingresos_trimestre,internet_tipo_nacional]
provinciales = [internet_accesos_velocidad_provincias,internet_penetracion_hogares,internet_tipo_provincias]
locales=[localidades_servicios]

In [81]:
# deshacerse de algunas columnas innecesarias
historico_velocidad_nacional.drop(columns='trimestre_d',inplace=True)
ingresos_trimestre.drop(columns='periodo',inplace=True)

In [82]:
df_nacionales = reduce(lambda left,right:pd.merge(left,right, on=['año','trimestre'],how='outer'),nacionales) # uso reduce para hacer un join de los dataframes

In [83]:
internet_tipo_provincias['trimestre']=internet_tipo_provincias.trimestre.astype(int)
internet_accesos_velocidad_provincias.rename(columns={'otros':'otras_velocidades'},inplace=True)
internet_tipo_provincias.drop(columns='total',inplace=True)

In [84]:
df_provinciales = reduce(lambda left,right:pd.merge(left,right,on=['año','trimestre','provincia'],how='outer'),provinciales)

In [87]:
df_locales = locales